In [1]:
import torch
from torch.utils.data import DataLoader, Dataset
from transformers import ElectraTokenizer, ElectraForSequenceClassification,  ElectraModel
from sklearn.model_selection import train_test_split
import pandas as pd 
from sklearn.preprocessing import LabelEncoder
import torch.nn as nn
import torch_xla.core.xla_model as xm
from torch.optim import AdamW


/usr/local/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/usr/local/lib/python3.10/site-packages/torch_xla/__init__.py:202: UserWarning: `tensorflow` can conflict with `torch-xla`. Prefer `tensorflow-cpu` when using PyTorch/XLA. To silence this warning, `pip uninstall -y tensorflow && pip install tensorflow-cpu`. If you are in a notebook environment such as Colab or Kaggle, restart your notebook runtime afterwards.
  warnings.warn(


In [2]:

review_train_df = pd.read_csv('/kaggle/input/review-train/review_sentiment.csv', index_col = 0)


positive_reviews = review_train_df[review_train_df['sentiment'] == 1]
augmented_reviews = positive_reviews.copy()
review_train_df = pd.concat([review_train_df, augmented_reviews], ignore_index=True)

print("데이터셋 크기:", review_train_df.shape)


데이터셋 크기: (1370, 2)


In [3]:
sent_encoder = LabelEncoder()

review_train_df['sentiment'] = sent_encoder.fit_transform(review_train_df['sentiment'])
train_texts, val_texts, train_labels, val_labels = train_test_split(
    review_train_df['review'], 
    review_train_df['sentiment'], 
    test_size=0.2, 
    random_state=1, 
    stratify=review_train_df['sentiment']
)

train_texts = train_texts.reset_index(drop=True)
train_labels = train_labels.reset_index(drop=True)

val_texts = val_texts.reset_index(drop=True)
val_labels = val_labels.reset_index(drop=True)
review_train_df

,review,sentiment
0,"물품 자체의 품질과 맛은 좋아서 정기적으로 주문하고 있습니다다만, 별점을 낮게 선택...",0
1,서방아이디로 가입하고 긴 세월동안 스마일클럽 회원비따위가 뭐라고 수시로 터져서 오고...,0
2,가격이 들쑥날쑥 처음부터 싸게 팔던가ㅡ가격을 일정하게 팔던가ㅡ가격이 지네들 맘대로네...,0
3,비비고 곰탕 2박스를 화요일 오전에 신청하면 다음날 수요일에 배송 된다 하여 목요일...,0
4,지금 장난 하나 딱 봐도 터져서 샌게 눈에 보이는데 그걸 테이프로 덕지덕지 감아서 ...,0
...,...,...
1365,이젠 자극적인 맛보다 순한 맛이 좋아요,1
1366,순한맛 맛있어요 면발 쫄깃,1
1367,이마트배송 원하는시간에 최고입니다!,1
1368,진라면이 리뉴얼 됐다고하더니 기존것보다 맵네요..,0


In [5]:
class SentimentDataset(Dataset):
    def __init__(self, texts, labels, tokenizer, max_length = 512):
        self.texts = texts
        self.labels = labels
        self.tokenizer = tokenizer
        self.max_length = max_length


    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        text = self.texts[idx]
        label = self.labels[idx]

        inputs = self.tokenizer(
            text,
            padding = 'max_length',
            truncation = True,
            max_length = self.max_length,
            return_tensors = 'pt'
        )

        input_ids = inputs['input_ids'].squeeze(0)
        attention_mask = inputs['attention_mask'].squeeze(0)

        return {'input_ids' : input_ids, 'attention_mask' : attention_mask, 
                'label' : torch.tensor(label).float() }


class LabelClassifier(nn.Module):

    def __init__(self, model):
        super(LabelClassifier, self).__init__()
        self.model = model
        self.sentiment_output = nn.Linear(model.config.hidden_size , 2)

    def forward(self, input_ids, attention_mask):
        outputs = self.model(input_ids = input_ids, attention_mask = attention_mask)
        sentiment_logits = self.sentiment_output(outputs.last_hidden_state[:, 0, :])
        sentiment_probs = torch.softmax(sentiment_logits, dim = -1)
        return sentiment_probs

In [6]:
tokenizer = ElectraTokenizer.from_pretrained('monologg/koelectra-base-v3-discriminator')
pretrained_model = ElectraModel.from_pretrained('monologg/koelectra-base-v3-discriminator')
model = LabelClassifier(pretrained_model)


train_dataset = SentimentDataset(train_texts, train_labels, tokenizer)
val_dataset = SentimentDataset(val_texts, val_labels, tokenizer)

train_loader = DataLoader(train_dataset, batch_size = 8 , shuffle = True)
val_loader = DataLoader(val_dataset, batch_size = 8, shuffle = True)


device = xm.xla_device()

model.to(device)

optimizer = AdamW(model.parameters(), lr = 2e-5)
loss_fn_sentiment = nn.CrossEntropyLoss()
epochs = 3
print(f"Using device: {device}")


E0000 00:00:1736400418.916073      13 common_lib.cc:818] Could not set metric server port: INVALID_ARGUMENT: Could not find SliceBuilder port 8471 in any of the 0 ports provided in `tpu_process_addresses`="local"
=== Source Location Trace: ===
learning/45eac/tfrc/runtime/common_lib.cc:483


Using device: xla:0


In [7]:

for epoch in range(epochs):
    model.train()
    total_loss = 0

    for batch in train_loader:
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        labels = batch['label'].long().to(device)

        optimizer.zero_grad()

        sentiment_probs = model(input_ids = input_ids, attention_mask = attention_mask)

        loss = loss_fn_sentiment(sentiment_probs, labels)
        loss.backward()

        xm.optimizer_step(optimizer)
        xm.mark_step()

        total_loss += loss.item()

    avg_train_loss = total_loss / len(train_loader)
    print(f"Epoch {epoch+1}/{epochs}, Average Training Loss: {avg_train_loss}")


    model.eval()
    val_loss = 0
    correct_labels = 0

    with torch.no_grad():
        for batch in val_loader:
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            labels = batch['label'].long().to(device)

            sentiment_probs = model(input_ids = input_ids, attention_mask = attention_mask)
            loss_sentiment = loss_fn_sentiment(sentiment_probs, labels)

            val_loss += loss_sentiment.item()
            preds_labels = torch.argmax(sentiment_probs, dim = 1)
            correct_labels += (preds_labels == labels).sum().item()

    avg_val_loss = val_loss / len(val_loader)
    accuracy_labels = correct_labels / len(val_loader.dataset)
    print(f"Validation Loss: {avg_val_loss}, Accuracy (Sentiment): {accuracy_labels}")



Epoch 1/3, Average Training Loss: 0.4573156855402202
Validation Loss: 0.35879618184907097, Accuracy (Sentiment): 0.948905109489051
Epoch 2/3, Average Training Loss: 0.34977419902808476
Validation Loss: 0.35538491436413355, Accuracy (Sentiment): 0.9562043795620438
Epoch 3/3, Average Training Loss: 0.342753036613882
Validation Loss: 0.3478247616972242, Accuracy (Sentiment): 0.9598540145985401


In [ ]:
#torch.save(model, '/kaggle/working/review_model_koelectra_complete.pt')
model = torch.load('/kaggle/working/review_model_koelectra_complete.pt')

In [12]:

torch.cuda.empty_cache()

In [8]:
def calculate_metrics_with_threshold(model, val_loader, device, threshold=0.45):
    model.eval()
    tp, fn, tn, fp = 0, 0, 0, 0

    with torch.no_grad():
        for batch in val_loader:
            # 데이터 로드
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            labels = batch['label'].long().to(device)

            # 모델 예측
            sentiment_probs = model(input_ids=input_ids, attention_mask=attention_mask)

            # Threshold 적용
            preds_labels = (sentiment_probs[:, 1] > threshold).long()  # Positive 클래스 확률 기준

            # Recall 및 Specificity 계산
            tp += ((preds_labels == 1) & (labels == 1)).sum().item()  # True Positives
            fn += ((preds_labels == 0) & (labels == 1)).sum().item()  # False Negatives
            tn += ((preds_labels == 0) & (labels == 0)).sum().item()  # True Negatives
            fp += ((preds_labels == 1) & (labels == 0)).sum().item()  # False Positives

    # Recall 계산
    recall = tp / (tp + fn) if (tp + fn) > 0 else 0.0

    # Specificity 계산
    specificity = tn / (tn + fp) if (tn + fp) > 0 else 0.0
    accuracy = (tn + tp) / (tn + tp + fp + fn)
    return recall, specificity, accuracy
for threshold in [0.7, 0.5, 0.3]:
    recall, specificity, accuracy = calculate_metrics_with_threshold(model, val_loader, device, threshold=threshold)
    print(f"Threshold: {threshold:.4f}, Recall: {recall:.4f}, Specificity: {specificity:.4f}, \
    Accuracy : {accuracy:.4f}")


Threshold: 0.7000, Recall: 0.9389, Specificity: 0.9860,     Accuracy : 0.9635
Threshold: 0.5000, Recall: 0.9389, Specificity: 0.9790,     Accuracy : 0.9599
Threshold: 0.3000, Recall: 0.9466, Specificity: 0.9790,     Accuracy : 0.9635


In [ ]:
model = torch.load('/kaggle/input/review_sentiment_koelectra/pytorch/default/1/review_model_koelectra')
tokenizer_path = '/kaggle/input/review_sentiment_koelectra/pytorch/default/1'

tokenizer = ElectraTokenizer.from_pretrained(tokenizer_path)

In [10]:
from transformers import BertTokenizer, BertModel
from torch.utils.data import DataLoader
import torch
import torch.nn as nn
from torch.optim import AdamW
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from torch.utils.data import Dataset

# Label Encoding
sent_encoder = LabelEncoder()
review_train_df['sentiment'] = sent_encoder.fit_transform(review_train_df['sentiment'])

# Train-test split
train_texts, val_texts, train_labels, val_labels = train_test_split(
    review_train_df['review'], 
    review_train_df['sentiment'], 
    test_size=0.2, 
    random_state=1, 
    stratify=review_train_df['sentiment']
)

train_texts = train_texts.reset_index(drop=True)
train_labels = train_labels.reset_index(drop=True)
val_texts = val_texts.reset_index(drop=True)
val_labels = val_labels.reset_index(drop=True)

# Dataset class
class SentimentDataset(Dataset):
    def __init__(self, texts, labels, tokenizer, max_length=512):
        self.texts = texts
        self.labels = labels
        self.tokenizer = tokenizer
        self.max_length = max_length

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        text = self.texts[idx]
        label = self.labels[idx]

        inputs = self.tokenizer(
            text,
            padding='max_length',
            truncation=True,
            max_length=self.max_length,
            return_tensors='pt'
        )

        input_ids = inputs['input_ids'].squeeze(0)
        attention_mask = inputs['attention_mask'].squeeze(0)

        return {'input_ids': input_ids, 'attention_mask': attention_mask, 
                'label': torch.tensor(label).float()}

# Model class
class LabelClassifier(nn.Module):
    def __init__(self, model):
        super(LabelClassifier, self).__init__()
        self.model = model
        self.sentiment_output = nn.Linear(model.config.hidden_size, 2)

    def forward(self, input_ids, attention_mask):
        outputs = self.model(input_ids=input_ids, attention_mask=attention_mask)
        sentiment_logits = self.sentiment_output(outputs.last_hidden_state[:, 0, :])
        sentiment_probs = torch.softmax(sentiment_logits, dim=-1)
        return sentiment_probs

# Define a function for training and evaluation
def train_and_evaluate(tokenizer_name, model_name):
    print(f"Training with {model_name}")
    tokenizer = BertTokenizer.from_pretrained(tokenizer_name)
    pretrained_model = BertModel.from_pretrained(model_name)
    model = LabelClassifier(pretrained_model)

    train_dataset = SentimentDataset(train_texts, train_labels, tokenizer)
    val_dataset = SentimentDataset(val_texts, val_labels, tokenizer)

    train_loader = DataLoader(train_dataset, batch_size=8, shuffle=True)
    val_loader = DataLoader(val_dataset, batch_size=8, shuffle=True)

    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model.to(device)

    optimizer = AdamW(model.parameters(), lr=2e-5)
    loss_fn_sentiment = nn.CrossEntropyLoss()

    epochs = 3

    for epoch in range(epochs):
        model.train()
        total_loss = 0

        for batch in train_loader:
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            labels = batch['label'].long().to(device)

            optimizer.zero_grad()

            sentiment_probs = model(input_ids=input_ids, attention_mask=attention_mask)

            loss = loss_fn_sentiment(sentiment_probs, labels)
            loss.backward()

            optimizer.step()

            total_loss += loss.item()

        avg_train_loss = total_loss / len(train_loader)
        print(f"Epoch {epoch + 1}/{epochs}, Average Training Loss: {avg_train_loss}")

        # Validation
        model.eval()
        val_loss = 0
        correct_labels = 0

        with torch.no_grad():
            for batch in val_loader:
                input_ids = batch['input_ids'].to(device)
                attention_mask = batch['attention_mask'].to(device)
                labels = batch['label'].long().to(device)

                sentiment_probs = model(input_ids=input_ids, attention_mask=attention_mask)
                loss_sentiment = loss_fn_sentiment(sentiment_probs, labels)

                val_loss += loss_sentiment.item()
                preds_labels = torch.argmax(sentiment_probs, dim=1)
                correct_labels += (preds_labels == labels).sum().item()

        avg_val_loss = val_loss / len(val_loader)
        accuracy_labels = correct_labels / len(val_loader.dataset)
        print(f"Validation Loss: {avg_val_loss}, Accuracy: {accuracy_labels}")

# Train with KoBERT
train_and_evaluate('monologg/kobert', 'monologg/kobert')


Training with monologg/kobert


The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'KoBertTokenizer'. 
The class this function is called from is 'BertTokenizer'.


Epoch 1/3, Average Training Loss: 0.6002283870738788
Validation Loss: 0.6148579640047891, Accuracy: 0.6897810218978102
Epoch 2/3, Average Training Loss: 0.5664916856445535
Validation Loss: 0.6021664065974099, Accuracy: 0.718978102189781
Epoch 3/3, Average Training Loss: 0.5576770423102553
Validation Loss: 0.5804229634148734, Accuracy: 0.7153284671532847


In [19]:

review_train_df = pd.read_csv('/kaggle/input/review-train/review_sentiment.csv', index_col = 0)
positive_reviews = review_train_df[review_train_df['sentiment'] == 1]
augmented_reviews = positive_reviews.copy()
review_train_df = pd.concat([review_train_df, augmented_reviews], ignore_index=True)


sent_encoder = LabelEncoder()

review_train_df['sentiment'] = sent_encoder.fit_transform(review_train_df['sentiment'])
train_texts, val_texts, train_labels, val_labels = train_test_split(
    review_train_df['review'], 
    review_train_df['sentiment'], 
    test_size=0.2, 
    random_state=1, 
    stratify=review_train_df['sentiment']
)

train_texts = train_texts.reset_index(drop=True)
train_labels = train_labels.reset_index(drop=True)

val_texts = val_texts.reset_index(drop=True)
val_labels = val_labels.reset_index(drop=True)# Label Encoding
sent_encoder = LabelEncoder()
review_train_df['sentiment'] = sent_encoder.fit_transform(review_train_df['sentiment'])

# Train-test split
train_texts, val_texts, train_labels, val_labels = train_test_split(
    review_train_df['review'], 
    review_train_df['sentiment'], 
    test_size=0.2, 
    random_state=1, 
    stratify=review_train_df['sentiment']
)

train_texts = train_texts.reset_index(drop=True)
train_labels = train_labels.reset_index(drop=True)
val_texts = val_texts.reset_index(drop=True)
val_labels = val_labels.reset_index(drop=True)

# Dataset class
class SentimentDataset(Dataset):
    def __init__(self, texts, labels, tokenizer, max_length=300):  # max_length 수정
        self.texts = texts
        self.labels = labels
        self.tokenizer = tokenizer
        self.max_length = max_length

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        text = self.texts[idx]
        label = self.labels[idx]

        inputs = self.tokenizer(
            text,
            padding='max_length',
            truncation=True,
            max_length=self.max_length,  # max_length 반영
            return_tensors='pt'
        )

        input_ids = inputs['input_ids'].squeeze(0)
        attention_mask = inputs['attention_mask'].squeeze(0)

        return {'input_ids': input_ids, 'attention_mask': attention_mask, 
                'label': torch.tensor(label).float()}

# Model class
class LabelClassifier(nn.Module):
    def __init__(self, model):
        super(LabelClassifier, self).__init__()
        self.model = model
        self.sentiment_output = nn.Linear(model.config.hidden_size, 2)

    def forward(self, input_ids, attention_mask):
        outputs = self.model(input_ids=input_ids, attention_mask=attention_mask)
        sentiment_logits = self.sentiment_output(outputs.last_hidden_state[:, 0, :])
        sentiment_probs = torch.softmax(sentiment_logits, dim=-1)
        return sentiment_probs

def train_and_evaluate(tokenizer_name, model_name, max_length):
    print(f"Training with {model_name}")
    tokenizer = BertTokenizer.from_pretrained(tokenizer_name)
    pretrained_model = BertModel.from_pretrained(model_name)
    model = LabelClassifier(pretrained_model)

    train_dataset = SentimentDataset(train_texts, train_labels, tokenizer, max_length=max_length)
    val_dataset = SentimentDataset(val_texts, val_labels, tokenizer, max_length=max_length)

    train_loader = DataLoader(train_dataset, batch_size=8, shuffle=True)
    val_loader = DataLoader(val_dataset, batch_size=8, shuffle=True)

    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model.to(device)

    optimizer = AdamW(model.parameters(), lr=2e-5)
    loss_fn_sentiment = nn.CrossEntropyLoss()

    epochs = 3

    for epoch in range(epochs):
        model.train()
        total_loss = 0

        for batch in train_loader:
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            labels = batch['label'].long().to(device)

            optimizer.zero_grad()

            sentiment_probs = model(input_ids=input_ids, attention_mask=attention_mask)

            loss = loss_fn_sentiment(sentiment_probs, labels)
            loss.backward()

            optimizer.step()

            total_loss += loss.item()

        avg_train_loss = total_loss / len(train_loader)
        print(f"Epoch {epoch + 1}/{epochs}, Average Training Loss: {avg_train_loss}")

        # Validation
        model.eval()
        val_loss = 0
        correct_labels = 0

        with torch.no_grad():
            for batch in val_loader:
                input_ids = batch['input_ids'].to(device)
                attention_mask = batch['attention_mask'].to(device)
                labels = batch['label'].long().to(device)

                sentiment_probs = model(input_ids=input_ids, attention_mask=attention_mask)
                loss_sentiment = loss_fn_sentiment(sentiment_probs, labels)

                val_loss += loss_sentiment.item()
                preds_labels = torch.argmax(sentiment_probs, dim=1)
                correct_labels += (preds_labels == labels).sum().item()

        avg_val_loss = val_loss / len(val_loader)
        accuracy_labels = correct_labels / len(val_loader.dataset)
        print(f"Validation Loss: {avg_val_loss}, Accuracy: {accuracy_labels}")

train_and_evaluate('beomi/kcbert-base', 'beomi/kcbert-base', max_length=300)


Training with beomi/kcbert-base
Epoch 1/3, Average Training Loss: 0.39658306502356316
Validation Loss: 0.36725474280469556, Accuracy: 0.945679012345679
Epoch 2/3, Average Training Loss: 0.34684646863655505
Validation Loss: 0.3544377316446865, Accuracy: 0.9580246913580247
Epoch 3/3, Average Training Loss: 0.3354789238845186
Validation Loss: 0.4070997512808033, Accuracy: 0.9037037037037037


In [20]:

review_train_df = pd.read_csv('/kaggle/input/review-train/review_sentiment.csv', index_col = 0)


positive_reviews = review_train_df[review_train_df['sentiment'] == 1]
augmented_reviews = positive_reviews.copy()
review_train_df = pd.concat([review_train_df, augmented_reviews], ignore_index=True)

sent_encoder = LabelEncoder()

review_train_df['sentiment'] = sent_encoder.fit_transform(review_train_df['sentiment'])
train_texts, val_texts, train_labels, val_labels = train_test_split(
    review_train_df['review'], 
    review_train_df['sentiment'], 
    test_size=0.2, 
    random_state=1, 
    stratify=review_train_df['sentiment']
)

train_texts = train_texts.reset_index(drop=True)
train_labels = train_labels.reset_index(drop=True)

val_texts = val_texts.reset_index(drop=True)
val_labels = val_labels.reset_index(drop=True)
review_train_df

class SentimentDataset(Dataset):
    def __init__(self, texts, labels, tokenizer, max_length = 512):
        self.texts = texts
        self.labels = labels
        self.tokenizer = tokenizer
        self.max_length = max_length


    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        text = self.texts[idx]
        label = self.labels[idx]

        inputs = self.tokenizer(
            text,
            padding = 'max_length',
            truncation = True,
            max_length = self.max_length,
            return_tensors = 'pt'
        )

        input_ids = inputs['input_ids'].squeeze(0)
        attention_mask = inputs['attention_mask'].squeeze(0)

        return {'input_ids' : input_ids, 'attention_mask' : attention_mask, 
                'label' : torch.tensor(label).float() }


class LabelClassifier(nn.Module):

    def __init__(self, model):
        super(LabelClassifier, self).__init__()
        self.model = model
        self.sentiment_output = nn.Linear(model.config.hidden_size , 2)

    def forward(self, input_ids, attention_mask):
        outputs = self.model(input_ids = input_ids, attention_mask = attention_mask)
        sentiment_logits = self.sentiment_output(outputs.last_hidden_state[:, 0, :])
        sentiment_probs = torch.softmax(sentiment_logits, dim = -1)
        return sentiment_probs


tokenizer = ElectraTokenizer.from_pretrained('monologg/koelectra-base-v3-discriminator')
pretrained_model = ElectraModel.from_pretrained('monologg/koelectra-base-v3-discriminator')
model = LabelClassifier(pretrained_model)


train_dataset = SentimentDataset(train_texts, train_labels, tokenizer)
val_dataset = SentimentDataset(val_texts, val_labels, tokenizer)

train_loader = DataLoader(train_dataset, batch_size = 8 , shuffle = True)
val_loader = DataLoader(val_dataset, batch_size = 8, shuffle = True)


device = xm.xla_device()

model.to(device)

optimizer = AdamW(model.parameters(), lr = 2e-5)
loss_fn_sentiment = nn.CrossEntropyLoss()
epochs = 3
print(f"Using device: {device}")



for epoch in range(epochs):
    model.train()
    total_loss = 0

    for batch in train_loader:
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        labels = batch['label'].long().to(device)

        optimizer.zero_grad()

        sentiment_probs = model(input_ids = input_ids, attention_mask = attention_mask)

        loss = loss_fn_sentiment(sentiment_probs, labels)
        loss.backward()

        xm.optimizer_step(optimizer)
        xm.mark_step()

        total_loss += loss.item()

    avg_train_loss = total_loss / len(train_loader)
    print(f"Epoch {epoch+1}/{epochs}, Average Training Loss: {avg_train_loss}")


    model.eval()
    val_loss = 0
    correct_labels = 0

    with torch.no_grad():
        for batch in val_loader:
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            labels = batch['label'].long().to(device)

            sentiment_probs = model(input_ids = input_ids, attention_mask = attention_mask)
            loss_sentiment = loss_fn_sentiment(sentiment_probs, labels)

            val_loss += loss_sentiment.item()
            preds_labels = torch.argmax(sentiment_probs, dim = 1)
            correct_labels += (preds_labels == labels).sum().item()

    avg_val_loss = val_loss / len(val_loader)
    accuracy_labels = correct_labels / len(val_loader.dataset)
    print(f"Validation Loss: {avg_val_loss}, Accuracy (Sentiment): {accuracy_labels}")


def calculate_metrics_with_threshold(model, val_loader, device, threshold=0.45):
    model.eval()
    tp, fn, tn, fp = 0, 0, 0, 0

    with torch.no_grad():
        for batch in val_loader:
            # 데이터 로드
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            labels = batch['label'].long().to(device)

            # 모델 예측
            sentiment_probs = model(input_ids=input_ids, attention_mask=attention_mask)

            # Threshold 적용
            preds_labels = (sentiment_probs[:, 1] > threshold).long()  # Positive 클래스 확률 기준

            # Recall 및 Specificity 계산
            tp += ((preds_labels == 1) & (labels == 1)).sum().item()  # True Positives
            fn += ((preds_labels == 0) & (labels == 1)).sum().item()  # False Negatives
            tn += ((preds_labels == 0) & (labels == 0)).sum().item()  # True Negatives
            fp += ((preds_labels == 1) & (labels == 0)).sum().item()  # False Positives

    # Recall 계산
    recall = tp / (tp + fn) if (tp + fn) > 0 else 0.0

    # Specificity 계산
    specificity = tn / (tn + fp) if (tn + fp) > 0 else 0.0
    accuracy = (tn + tp) / (tn + tp + fp + fn)
    return recall, specificity, accuracy
for threshold in [0.7, 0.5, 0.3]:
    recall, specificity, accuracy = calculate_metrics_with_threshold(model, val_loader, device, threshold=threshold)
    print(f"Threshold: {threshold:.4f}, Recall: {recall:.4f}, Specificity: {specificity:.4f}, \
    Accuracy : {accuracy:.4f}")

Using device: xla:0
Epoch 1/3, Average Training Loss: 0.40976559440490645
Validation Loss: 0.34429242505746727, Accuracy (Sentiment): 0.9703703703703703
Epoch 2/3, Average Training Loss: 0.3383751880065561
Validation Loss: 0.34911946747817246, Accuracy (Sentiment): 0.9654320987654321
Epoch 3/3, Average Training Loss: 0.35112667553530535
Validation Loss: 0.3498621635577258, Accuracy (Sentiment): 0.9654320987654321
Threshold: 0.7000, Recall: 0.9847, Specificity: 0.9301,     Accuracy : 0.9654
Threshold: 0.5000, Recall: 0.9847, Specificity: 0.9301,     Accuracy : 0.9654
Threshold: 0.3000, Recall: 0.9847, Specificity: 0.9231,     Accuracy : 0.9630


In [21]:

review_train_df = pd.read_csv('/kaggle/input/review-train/review_sentiment.csv', index_col = 0)

sent_encoder = LabelEncoder()

review_train_df['sentiment'] = sent_encoder.fit_transform(review_train_df['sentiment'])
train_texts, val_texts, train_labels, val_labels = train_test_split(
    review_train_df['review'], 
    review_train_df['sentiment'], 
    test_size=0.2, 
    random_state=1, 
    stratify=review_train_df['sentiment']
)

train_texts = train_texts.reset_index(drop=True)
train_labels = train_labels.reset_index(drop=True)

val_texts = val_texts.reset_index(drop=True)
val_labels = val_labels.reset_index(drop=True)
review_train_df

class SentimentDataset(Dataset):
    def __init__(self, texts, labels, tokenizer, max_length = 512):
        self.texts = texts
        self.labels = labels
        self.tokenizer = tokenizer
        self.max_length = max_length


    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        text = self.texts[idx]
        label = self.labels[idx]

        inputs = self.tokenizer(
            text,
            padding = 'max_length',
            truncation = True,
            max_length = self.max_length,
            return_tensors = 'pt'
        )

        input_ids = inputs['input_ids'].squeeze(0)
        attention_mask = inputs['attention_mask'].squeeze(0)

        return {'input_ids' : input_ids, 'attention_mask' : attention_mask, 
                'label' : torch.tensor(label).float() }


class LabelClassifier(nn.Module):

    def __init__(self, model):
        super(LabelClassifier, self).__init__()
        self.model = model
        self.sentiment_output = nn.Linear(model.config.hidden_size , 2)

    def forward(self, input_ids, attention_mask):
        outputs = self.model(input_ids = input_ids, attention_mask = attention_mask)
        sentiment_logits = self.sentiment_output(outputs.last_hidden_state[:, 0, :])
        sentiment_probs = torch.softmax(sentiment_logits, dim = -1)
        return sentiment_probs


tokenizer = ElectraTokenizer.from_pretrained('monologg/koelectra-base-v3-discriminator')
pretrained_model = ElectraModel.from_pretrained('monologg/koelectra-base-v3-discriminator')
model = LabelClassifier(pretrained_model)


train_dataset = SentimentDataset(train_texts, train_labels, tokenizer)
val_dataset = SentimentDataset(val_texts, val_labels, tokenizer)

train_loader = DataLoader(train_dataset, batch_size = 8 , shuffle = True)
val_loader = DataLoader(val_dataset, batch_size = 8, shuffle = True)


device = xm.xla_device()

model.to(device)

optimizer = AdamW(model.parameters(), lr = 2e-5)
loss_fn_sentiment = nn.CrossEntropyLoss()
epochs = 3
print(f"Using device: {device}")



for epoch in range(epochs):
    model.train()
    total_loss = 0

    for batch in train_loader:
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        labels = batch['label'].long().to(device)

        optimizer.zero_grad()

        sentiment_probs = model(input_ids = input_ids, attention_mask = attention_mask)

        loss = loss_fn_sentiment(sentiment_probs, labels)
        loss.backward()

        xm.optimizer_step(optimizer)
        xm.mark_step()

        total_loss += loss.item()

    avg_train_loss = total_loss / len(train_loader)
    print(f"Epoch {epoch+1}/{epochs}, Average Training Loss: {avg_train_loss}")


    model.eval()
    val_loss = 0
    correct_labels = 0

    with torch.no_grad():
        for batch in val_loader:
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            labels = batch['label'].long().to(device)

            sentiment_probs = model(input_ids = input_ids, attention_mask = attention_mask)
            loss_sentiment = loss_fn_sentiment(sentiment_probs, labels)

            val_loss += loss_sentiment.item()
            preds_labels = torch.argmax(sentiment_probs, dim = 1)
            correct_labels += (preds_labels == labels).sum().item()

    avg_val_loss = val_loss / len(val_loader)
    accuracy_labels = correct_labels / len(val_loader.dataset)
    print(f"Validation Loss: {avg_val_loss}, Accuracy (Sentiment): {accuracy_labels}")


def calculate_metrics_with_threshold(model, val_loader, device, threshold=0.45):
    model.eval()
    tp, fn, tn, fp = 0, 0, 0, 0

    with torch.no_grad():
        for batch in val_loader:
            # 데이터 로드
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            labels = batch['label'].long().to(device)

            # 모델 예측
            sentiment_probs = model(input_ids=input_ids, attention_mask=attention_mask)

            # Threshold 적용
            preds_labels = (sentiment_probs[:, 1] > threshold).long()  # Positive 클래스 확률 기준

            # Recall 및 Specificity 계산
            tp += ((preds_labels == 1) & (labels == 1)).sum().item()  # True Positives
            fn += ((preds_labels == 0) & (labels == 1)).sum().item()  # False Negatives
            tn += ((preds_labels == 0) & (labels == 0)).sum().item()  # True Negatives
            fp += ((preds_labels == 1) & (labels == 0)).sum().item()  # False Positives

    # Recall 계산
    recall = tp / (tp + fn) if (tp + fn) > 0 else 0.0

    # Specificity 계산
    specificity = tn / (tn + fp) if (tn + fp) > 0 else 0.0
    accuracy = (tn + tp) / (tn + tp + fp + fn)
    return recall, specificity, accuracy
for threshold in [0.6, 0.5, 0.4]:
    recall, specificity, accuracy = calculate_metrics_with_threshold(model, val_loader, device, threshold=threshold)
    print(f"Threshold: {threshold:.4f}, Recall: {recall:.4f}, Specificity: {specificity:.4f}, \
    Accuracy : {accuracy:.4f}")

Using device: xla:0
Epoch 1/3, Average Training Loss: 0.4756864235348945
Validation Loss: 0.3805523668016706, Accuracy (Sentiment): 0.9416058394160584
Epoch 2/3, Average Training Loss: 0.3497067285280158
Validation Loss: 0.3811680836336953, Accuracy (Sentiment): 0.9416058394160584
Epoch 3/3, Average Training Loss: 0.34342056904396
Validation Loss: 0.37136587500572205, Accuracy (Sentiment): 0.9416058394160584
Threshold: 0.9000, Recall: 0.8779, Specificity: 0.9860,     Accuracy : 0.9343
Threshold: 0.8000, Recall: 0.8779, Specificity: 0.9860,     Accuracy : 0.9343
Threshold: 0.7000, Recall: 0.8779, Specificity: 0.9860,     Accuracy : 0.9343


In [ ]:
for threshold in [0.6, 0.5, 0.4]:
    recall, specificity, accuracy = calculate_metrics_with_threshold(model, val_loader, device, threshold=threshold)
    print(f"Threshold: {threshold:.4f}, Recall: {recall:.4f}, Specificity: {specificity:.4f}, \
    Accuracy : {accuracy:.4f}")

In [22]:
def find_misclassified_texts(model, val_loader, device, threshold=0.5):
    model.eval()
    misclassified_positive_as_negative = []  # 긍정을 부정으로 분류
    misclassified_negative_as_positive = []  # 부정을 긍정으로 분류

    with torch.no_grad():
        for batch in val_loader:
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            labels = batch['label'].long().to(device)

            sentiment_probs = model(input_ids=input_ids, attention_mask=attention_mask)

            # Threshold 적용
            preds_labels = (sentiment_probs[:, 1] > threshold).long()

            # 원본 텍스트와 잘못 분류된 텍스트 확인
            for i, (pred, label) in enumerate(zip(preds_labels, labels)):
                original_text = tokenizer.decode(batch['input_ids'][i], skip_special_tokens=True)

                # 긍정을 부정으로 잘못 분류
                if label == 1 and pred == 0:
                    misclassified_positive_as_negative.append(original_text)

                # 부정을 긍정으로 잘못 분류
                elif label == 0 and pred == 1:
                    misclassified_negative_as_positive.append(original_text)

    return misclassified_positive_as_negative, misclassified_negative_as_positive


# 잘못 분류된 텍스트 확인
misclassified_positive_as_negative, misclassified_negative_as_positive = find_misclassified_texts(
    model, val_loader, device, threshold=0.5
)

# 출력
print("긍정을 부정으로 분류한 텍스트:")
for text in misclassified_positive_as_negative:
    print(text)

print("\n부정을 긍정으로 분류한 텍스트:")
for text in misclassified_negative_as_positive:
    print(text)


긍정을 부정으로 분류한 텍스트:
가격이 많이 올랐네요 ~ ㅎ
기승전 진라면 뭐 먹을까 고민하다 삼
예전 저렴할때 생각하면 많이 올랐네요
진라면은 제 입에는 안맞는거 같아요! 특유의 냄새가 있네요!
할인하네요 두개사니
소세지 맛이 바꼈네요? 전에는 짜기만 해서 데쳐먹었는데 생으로 먹어도 안짜고 너무 맛있어요
양이 생각보다 꽤많아요
ㅎ
불맛나는 비엔나 어차피 햄은 몸에 좋지 않으니 맛있는거라도 …
매운걸 못먹는 아이가 이 라면만 먹어요
항상 배송도 빠르고 제품 손상 된 거 없이 잘 와요!! 계란두 하나도 안깨져서 오네요 ㅎ
가격이저렴해서 구입햇는데 육수대용으로 잘사용합니다
아이가 매운걸 못먹어서 늘 순한맛만 사네요 근데 먹다보니 이것만한것도 없는듯 ㅋ
아직 먹어보진 못했지만 도시락 반찬으로 샀어요

부정을 긍정으로 분류한 텍스트:
평많은곳에서 주문하세여 ~ 경험했다ㅋㅋ
빠른 배송 감사합니다
